## Commets from reviewer

<span style="color:green"> Hi! I really enjoyed reading your work. Very clean and smart work! Your project is accepted. I left you some technical comments along the notebook, please, read them) Hope they will help you to explore other methods) Have a good one :) </span>

## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/credit_scoring_eng.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Conclusion

The dataset has 12 columns all of them have 21525 rows except two columns, 'total_income' and 'days_employed'. It is possible that values are missed in one row. The missing values are probably MAR (missing at random) the values are connected to each other for example a client with no days employed might also have no income data.

### Step 2. Data preprocessing

### Processing missing values

In [2]:
data['days_employed'] = data['days_employed'].apply(abs)
data['children'] = data['children'].apply(abs)
median_days_employed = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(median_days_employed)

In [3]:
# low_income_median - the median of low income clients
# low_middle_income_median - the median of low middle income clients
# high_income_median - the median of high income clients

low_income_median = data[data['total_income'] < 30000]['total_income'].median()
low_middle_income_median = data[(data['total_income'] >= 30000) & (data['total_income'] < 50000)]['total_income'].median()
average_income_median = data[(data['total_income'] >= 50000) & (data['total_income'] > 88000)]['total_income'].median()
high_income_median = data[data['total_income'] >= 88000]['total_income'].median()

In [4]:
# "for" loop for filling out the missing values of total_income column by the medians
for row in data['total_income']:
    if row < 30000:
        data['total_income'] = data['total_income'].fillna(low_income_median)
    if 30000 <= row < 50000:
        data['total_income'] = data['total_income'].fillna(low_middle_income_median)
    if 50000 <= row < 88000:
        data['total_income'] = data['total_income'].fillna(average_income_median)
    if row >= 88000:
        data['total_income'] = data['total_income'].fillna(high_income_median)

<span style="color:green">Nice) You can also try using .transform() method for filling in missing values by groups https://towardsdatascience.com/using-pandas-transform-and-apply-to-deal-with-missing-data-on-a-group-level-cb6ccf060531</span>


### Conclusion

Using mean values for filling out the missing values can distort the results because some values can be outliers (too big or too small, this can affect the mean value). That is why the median is used to avoid potential errors. Clients are grouped according to their total income data to four distinct groups: low income, low middle income, average and high. The median is found for each individual group and then used to fill out the missing values of these groups.

### Data type replacement

In [5]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)

### Conclusion

Astype method is used because it can convert the values to integers.

### Processing duplicates

In [6]:
data['education'] = data['education'].str.lower()
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum() 
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding


### Conclusion

The 'education' column had the same data written with lower letters and upper letter, it created duplicates. The data in the column was changed to lower letters and the duplicates were dropped. 

### Categorizing Data

In [7]:
print(data['purpose'].unique())

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to buy a car' 'housing renovation'
 'going to university']


In [8]:
#lemmatization function
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
def lemma_purpose(row):
    lemma_purposes = m.lemmatize(row['purpose'])
    if 'estate' in lemma_purposes:
         return 'purchase/building of a property'
    if 'house' in lemma_purposes:
        return 'purchase/building of a property'
    if 'property' in lemma_purposes:
        return 'purchase/building of a property'
    if 'car' in lemma_purposes:
        return 'purchase a car'
    if 'wedding' in lemma_purposes:
        return 'wedding'
    if 'education' in lemma_purposes:
        return 'education'
    if 'university' in lemma_purposes:
        return 'education'
    return 'other'

<span style="color:green">Nice job here) For the function you can use smth like :</span>
```
if 'estate' in purpose or 'property' in purpose:
    return 4
```
<span style="color:green">No need to that in every row)) Or create a list for each category and do smth like https://www.geeksforgeeks.org/python-check-if-a-list-exists-in-given-list-of-lists/ </span>

In [9]:
#purposes of loans are classified after categorizing them to just four categories
data['purpose_classified'] = data.apply(lemma_purpose, axis=1)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_classified
0,1,8437,42,bachelor's degree,0,married,0,F,employee,0,40620,purchase of the house,purchase/building of a property
1,1,4024,36,secondary education,1,married,0,F,employee,0,17932,car purchase,purchase a car
2,0,5623,33,secondary education,1,married,0,M,employee,0,23341,purchase of the house,purchase/building of a property
3,3,4124,32,secondary education,1,married,0,M,employee,0,42820,supplementary education,education
4,0,340266,53,secondary education,1,civil partnership,1,F,retiree,0,25378,to have a wedding,wedding


In [10]:
#this dictionary contains information about debts and purposes of loans
purpose_debt = data.pivot_table(index=['purpose_classified'], columns='debt', values='dob_years', aggfunc='count').reset_index()
purpose_debt

debt,purpose_classified,0,1
0,education,3274,331
1,other,2579,212
2,purchase a car,3469,359
3,purchase/building of a property,8253,653
4,wedding,2138,186


In [11]:
def income_classified(row):
    if row['total_income'] >= 88000:
        return 'high income'
    if 50000 <= row['total_income'] < 88000:
        return 'average income'
    if  30000 <= row['total_income'] < 50000:
        return 'low middle income'
    return 'low income'

In [12]:
data['income_classified'] = data.apply(income_classified, axis=1)

In [13]:
income_debt= data.pivot_table(index=['income_classified'], columns='debt', values='dob_years', aggfunc='count').reset_index()
income_debt

debt,income_classified,0,1
0,average income,1083,83
1,high income,145,9
2,low income,12297,1135
3,low middle income,6188,514


In [14]:
kids_debt = data.pivot_table(index=['children'], columns='debt', values='dob_years', aggfunc='count').reset_index()
kids_debt = kids_debt.fillna(0)
kids_debt

debt,children,0,1
0,0,13028.0,1063.0
1,1,4410.0,445.0
2,2,1858.0,194.0
3,3,303.0,27.0
4,4,37.0,4.0
5,5,9.0,0.0
6,20,68.0,8.0


In [15]:
marital_debt = data.pivot_table(index='family_status', columns='debt', values='dob_years', aggfunc='count').reset_index()
marital_debt

debt,family_status,0,1
0,civil partnership,3763,388
1,divorced,1110,85
2,married,11408,931
3,unmarried,2536,274
4,widow / widower,896,63


<span style="color:green">Everything is good here! </span>

### Conclusion

The purposes were the same but written in different forms like "buying a house" and "purchase a house". It is not convenient because two same purposes end up being categorized separately. That is why a new column with four distinct purposes was created. Lemmatization was used for these purposes. Overall, four dictionaries were created purpose/debt, having children/debt, income/debt, marital status/ debt.

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [16]:
kids_debt['debtor rate %'] = ((kids_debt[1] / (kids_debt[0] + kids_debt[1]))*100).astype(int)
kids_debt

debt,children,0,1,debtor rate %
0,0,13028.0,1063.0,7
1,1,4410.0,445.0,9
2,2,1858.0,194.0,9
3,3,303.0,27.0,8
4,4,37.0,4.0,9
5,5,9.0,0.0,0
6,20,68.0,8.0,10


### Conclusion

In order to identify the relation of having children to debt among clients, the number of debtors was divided by the total number of clients of each category. As seen from the table, the debtor's rates are not high and approximately same in every category. This means that there is no obvious relation between having children and on time loan repayment. The are no debtors among clients with 5 children.

- Is there a relation between marital status and repaying a loan on time?

In [17]:
marital_debt['debtor rate %'] = ((marital_debt[1] / (marital_debt[0] + marital_debt[1])*100)).astype(int)
marital_debt                                          

debt,family_status,0,1,debtor rate %
0,civil partnership,3763,388,9
1,divorced,1110,85,7
2,married,11408,931,7
3,unmarried,2536,274,9
4,widow / widower,896,63,6


### Conclusion

In order to identify the relation of marital status to debt among clients, the number of debtors was divided by the total number of clients of each category. As seen from the table, the debtor's rates are not high and approximately same in every category. This means that there is no obvious relation between family status and on time loan repayment. Surprisingly, widow/widowers have the smallest debtor rate.

- Is there a relation between income level and repaying a loan on time?

In [18]:
income_debt['debtor rate %'] = ((income_debt[1] / (income_debt[0] + income_debt[1]))*100).astype(int)
income_debt

debt,income_classified,0,1,debtor rate %
0,average income,1083,83,7
1,high income,145,9,5
2,low income,12297,1135,8
3,low middle income,6188,514,7


### Conclusion

In order to identify the relation of income to debt among clients, the number of debtors was divided by the total number of clients of each category. As seen from the table, the debtor's rates are not high and approximately same in every category. This means that there is no obvious relation between income and on time loan repayment. Among all of the clients, those with high income have the best repayment rate, but people with high income are only a small part of bank clients.

- How do different loan purposes affect on-time repayment of the loan?

In [19]:
purpose_debt['debtor rate %'] = ((purpose_debt[1] / (purpose_debt[0] + purpose_debt[1]))*100).astype(int)
purpose_debt

debt,purpose_classified,0,1,debtor rate %
0,education,3274,331,9
1,other,2579,212,7
2,purchase a car,3469,359,9
3,purchase/building of a property,8253,653,7
4,wedding,2138,186,8


### Conclusion

In order to identify the relation of loan purposes to debt among clients, the number of debtors was divided by the total number of clients of each category. As seen from the table, the debtor's rates are not high and approximately same in every category. This means that there is no obvious relation between loan purposes and on time loan repayment. Clients with the purpose of buying or building a property have better on-time repayment rates.

### Step 4. General conclusion

In general, there is no obvious relation between marital status/having children/total income or even loan purpose and on-time repayment. The analysis identified the portrait of an ideal bank client. The ideal client is a widow/widower with high income but no children and probably has a purpose of buying of building a house. The bank can also consider married people with no children and average or low middle income.

<span style="color:green">Step 3 and general conclusion are great. Good job! </span>


### Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [ ]  file examined;
- [ ]  missing values defined;
- [ ]  missing values are filled;
- [ ]  an explanation of which missing value types were detected;
- [ ]  explanation for the possible causes of missing values;
- [ ]  an explanation of how the blanks are filled;
- [ ]  replaced the real data type with an integer;
- [ ]  an explanation of which method is used to change the data type and why;
- [ ]  duplicates deleted;
- [ ]  an explanation of which method is used to find and remove duplicates;
- [ ]  description of the possible reasons for the appearance of duplicates in the data;
- [ ]  data is categorized;
- [ ]  an explanation of the principle of data categorization;
- [ ]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [ ]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [ ]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [ ]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [ ]  conclusions are present on each stage;
- [ ]  a general conclusion is made.